In [1]:
import json
import pickle
import nltk
import random
import numpy as np

from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

nltk.download( 'punkt')
nltk.download( 'wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
# Parte 2
# inicializaremos nossa lista de palavras, classes, documentos e
# definimos quais palavras serão ignoradas
words = []
documents = []
# é feita a leitura do arquivo intents.json e transformado em json
intents = json.loads(open('intents.json').read())
# adicionamos as tags em nossa lista de classes
classes = [i['tag'] for i in intents['intencoes']]
ignore_words = ["!", "@", "#", "$", "%", "*", "?"]

# percorremos nosso array de objetos
for intent in intents['intencoes']:
    for pattern in intent['patterns']:
        # com ajuda no nltk fazemos aqui a tokenizaçao dos patterns
        # e adicionamos na lista de palavras
        word = nltk.word_tokenize(pattern)
        words.extend(word)

        # adiciona aos documentos para identificarmos a tag para a mesma
        documents.append((word, intent['tag']))

In [25]:
# Parte 3
# lematizamos as palavras ignorando as palavras da lista ignore_words
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# classificamos nossas listas
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# salvamos as palavras e classes nos arquivos pkl
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [26]:
# Parte 4
# Criar base de dados para o treinamento
training = []
output_empty = [0] * len(classes)
for document in documents:
    # inicializamos o saco de palavras
    bag = []
    # listamos as palavras do pattern
    pattern_words = document[0]
    # lematizamos cada palavra
    # na tentativa de representar palavras relacionadas
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # criamos nosso conjunto de palavras com 1,
    # se a correspondência de palavras for encontrada no padrão atual
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output_row atuará como uma chave para a lista,
    # onde a saída será 0 para cada tag e 1 para a tag atual
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# embaralhamos nosso conjunto de treinamentos e transformamos em numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

# criamos lista de treino sendo x os patterns e y as intenções
x = list(training[:, 0])
y = list(training[:, 1])

# Salvar x e y como arquivos pickle
pickle.dump(x, open('x.pkl', 'wb'))
pickle.dump(y, open('y.pkl', 'wb'))

In [27]:
# criamos nosso conjunto de palavras com 1,
# se a correspondência de palavras for encontrada no padrão atual
for word in words:
    bag.append(1) if word in pattern_words else bag.append(0)

# Create output row
output_row = list(output_empty)
output_row[classes.index(document[1])] = 1

# Create new_element with the modified dimension
new_element = np.array([bag, output_row], dtype=object)
new_element = new_element[np.newaxis, :]  # Add an extra dimension

# Concatenate new_element with training
training = np.concatenate((training, new_element), axis=0)

# Shuffle and convert training to numpy array
random.shuffle(training)
training = np.array(training)

# Create x and y lists for training
x = list(training[:, 0])
y = list(training[:, 1])

In [28]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def generate_pdf(data_lists):
    pdf_filename = "data_lists.pdf"

    # Inicializar o objeto Canvas
    c = canvas.Canvas(pdf_filename, pagesize=letter)

    # Definir as posições iniciais para escrever as listas
    x = 50
    y = 750

    # Escrever a lista de palavras (words)
    c.setFont("Helvetica", 12)
    c.drawString(x, y, "Lista de Palavras (words):")
    y -= 20
    for word in data_lists["words"]:
        c.drawString(x, y, word)
        y -= 15

    # Escrever a lista de classes
    y -= 20
    c.drawString(x, y, "Lista de Classes:")
    y -= 20
    for class_item in data_lists["classes"]:
        c.drawString(x, y, class_item)
        y -= 15

    # Escrever a lista de dados de treino X
    y -= 20
    c.drawString(x, y, "Lista de Dados de Treino X:")
    y -= 20
    for x_item in data_lists["x"]:
        c.drawString(x, y, str(x_item))
        y -= 15

    # Escrever a lista de alvos de treino Y
    y -= 20
    c.drawString(x, y, "Lista de Alvos de Treino Y:")
    y -= 20
    for y_item in data_lists["y"]:
        c.drawString(x, y, str(y_item))
        y -= 15

    # Salvar e fechar o arquivo PDF
    c.save()

# Obter as listas de dados
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
x = pickle.load(open('x.pkl', 'rb'))
y = pickle.load(open('y.pkl', 'rb'))

# Criar um dicionário com as listas de dados
data_lists = {
    "words": words,
    "classes": classes,
    "x": x,
    "y": y,
}

# Gerar o PDF com as listas de dados
generate_pdf(data_lists)